In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import gradio as gr
import os
import torch
from sentence_transformers import SentenceTransformer, util

device = "cuda" if torch.cuda.is_available() else "cpu"
model_similarity = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

In [ ]:
dataset = load_dataset("Juliofc/Conversaciones_terapeuticas_espanol_test")total = []
for i in dataset["train"]["Chat"]:
    total.append(len(i))

In [ ]:
x = []
dataset = load_dataset("Juliofc/Conversaciones_terapeuticas_espanol_test")
for i in range(len(dataset["train"]["Chat"])):
    sin_role = dataset["train"]["Chat"][i][1:]
    x.append(sin_role)

df = pd.DataFrame({"Chat": x})
dataset = Dataset.from_pandas(df)

In [ ]:
# Configuración inicial
tokenizer = AutoTokenizer.from_pretrained("somosnlp/chaterapia_model")
model_base = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it").to(device)
model_base.resize_token_embeddings(len(tokenizer))
model_with_adapter = PeftModel.from_pretrained(model_base, "somosnlp/chaterapia_model").to(device)

CHAT_TEMPLATE= """{% for message in messages %}
    {% if message['role'] == 'user' %}
        {{'<user> ' + message['content'].strip() + ' </user>' }}
    {% elif message['role'] == 'system' %}
        {{'<system>\\n' + message['content'].strip() + '\\n</system>\\n\\n' }}
    {% elif message['role'] == 'assistant' %}
        {{ message['content'].strip() + ' </assistant>' + eos_token }}
    {% elif message['role'] == 'input' %}
        {{'<input> ' + message['content'] + ' </input>' }}
    {% endif %}
{% endfor %}""" # Asegúrate de usar tu CHAT_TEMPLATE aquí
tokenizer.chat_template = CHAT_TEMPLATE

def generate_response(user_input):
    # Preparación del input para el modelo
    user_input = tokenizer.apply_chat_template(user_input, tokenize=False)
    input_tokens = tokenizer(user_input, return_tensors='pt', padding=True).to(device)#, truncation=True).to(device)#, max_length=1024).to(device)
    
    # Generación de la respuesta del modelo
    output_tokens = model_with_adapter.generate(**input_tokens, max_new_tokens = 1024, pad_token_id=tokenizer.eos_token_id)#, top_k=50, do_sample=True, top_p=0.95, temperature=0.7)
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
    # Extracción de la respuesta generada
    last_us = generated_text.rfind("</user>") + len("</user>")
    last_as = generated_text.rfind("</assistant>")
    generated_text = generated_text[last_us:last_as].strip()
    return generated_text

In [ ]:
def find_similarity(Chatbot_response, Real_response):
    # Compute embedding for both lists
    embeddings1 = model_similarity.encode(Chatbot_response, convert_to_tensor=True)
    embeddings2 = model_similarity.encode(Real_response, convert_to_tensor=True)

    # Compute cosine-similarities
    cosine_score = util.cos_sim(embeddings1, embeddings2)
    metric = round(float(cosine_score), 4)
    return metric

In [ ]:
total_metrics = []
for conv in dataset["Chat"]:
    conv_metrics={}
    inter = 1
    for chat in conv:
        if chat["role"] == "assistant":
            conv_metrics[str(inter)] = 0
            inter = inter + 1
    total_metrics.append(conv_metrics)

In [ ]:
ind= 1
for conv in range(len(dataset["Chat"])):
    print(ind)
    ind = ind+1
    conv_simil = []
    for inter in range(len(dataset["Chat"][conv])):
        if inter%2==1:
            try:
                response = generate_response(dataset["Chat"][conv][0:inter+1])
                real = dataset["Chat"][conv][inter]["content"]
                simil = find_similarity(response, real)
                conv_simil.append(simil)
            except:
                print("error")
                conv_simil.append(None)
        else:
            continue
    for i in range(len(conv_simil)):
        total_metrics[conv][str(i+1)] = conv_simil[i]
